## Tutorial: Create an Isolated VLAN and Servers

This notebook demonstrates how to create isolated VLANs and attach servers. Works after creating networks with corsa-eval-tacc


#### Modules 

- [Reserve a Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

In [ ]:
import json
import os
import chi
import time

from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *

#Config with your project and site
chi.set('project_name', 'CICI SAFE') # Replace with your project name
chi.set('region_name', 'CHI@TACC')     # Optional, defaults to 'CHI@UC'

#Insert keypair name
key_name = 'dukepc'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")

suffix = "v4"

vfc_name = username + "vfc" + suffix

server_name1 = username+'Server1' + suffix
network_name1 = vfc_name+'Net1' + suffix
subnet_name1 = username+'subnet1' + suffix
router_name1 = username+'Router1' + suffix
subnet1 = "192.168.110.1/24"

network_name3 = vfc_name+'ExoGENINet3' + suffix
subnet_name3 = username+'subnet3' + suffix
router_name3 = username+'Router3' + suffix
subnet3 = "192.168.160.1/24"

sdn_controller_ip = "24.163.82.116"
#sdn_controller_ip = "129.114.109.117"
sdn_controller_port = "6633"

node_lease_name = username + 'vfcnodes' + suffix
net_lease_name1= username+'vfcevalnet1' + suffix
net_lease_name3= username+'vfcevalnet3' + suffix

#Server attributes
image_name='CC-Ubuntu16.04'
flavor_name='baremetal'
node_type="compute_skylake"
server_count=1
physical_network="physnet1"
exogeni_network="exogeni"

DAYS = 7



#### Create a Lease

In [ ]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.

start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=DAYS)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
#add_network_reservation(reservation_list, network_name=network_name1, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=physical_network)
#add_network_reservation(reservation_list, network_name=network_name2, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=physical_network)
#add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=node_lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])


In [ ]:
# Reserve networks
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=DAYS)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name1, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=physical_network)
#add_network_reservation(reservation_list, network_name=network_name2, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=physical_network)
#add_network_reservation(reservation_list, network_name=network_name3, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=exogeni_network)
#add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=net_lease_name1, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])



In [ ]:
# Reserve networks
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=DAYS)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
#add_node_reservation(reservation_list, count=server_count, node_type=node_type)
#add_network_reservation(reservation_list, network_name=network_name1, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=physical_network)
#add_network_reservation(reservation_list, network_name=network_name2, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=physical_network)
add_network_reservation(reservation_list, network_name=network_name3, of_controller_ip = sdn_controller_ip, of_controller_port = sdn_controller_port, vswitch_name = vfc_name, physical_network=exogeni_network)
#add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=net_lease_name3, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])



#### Get the Reservations

In [ ]:
#Get the lease by name
lease = get_lease_by_name(node_lease_name)

#Print the lease info
print(json.dumps(lease, indent=2))

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
#network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', lease['reservations']))[0]['id']
#floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)

#print("network_reservation_id: " + network_reservation_id)
#print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Configute the Network

In [ ]:
time.sleep(90)

network1 = get_network_by_name(network_name1)
network_id1 = network1['id']
print('Network ID1: ' + str(network_id1))
network_vlan1 = network1['provider:segmentation_id']
print('network_vlan1: ' + str(network_vlan1))

In [ ]:
subnet1 = add_subnet(subnet_name1, network_name1, cidr = subnet1)
router1 = create_router(router_name1, "public")
attach_router_to_subnet(router_name=router_name1, subnet_name=subnet_name1)

In [ ]:
network3 = get_network_by_name(network_name3)
print(network_name3)
network_id3 = network3['id']
print('Network ID3: ' + str(network_id3))
network_vlan3 = network3['provider:segmentation_id']
print('network_vlan3: ' + str(network_vlan3))

#### Start the Server

In [ ]:
#create the server
server1 = create_server(server_name1, 
                       reservation_id=compute_reservation_id, 
                       key_name=key_name, 
                       network_name=network_name1, 
                       image_name=image_name, 
                       flavor_name=flavor_name)

#### Associate the Floating IP   (TODO: need to find floating_ip from the reservation that was just made)

In [ ]:
time.sleep(1200)

In [ ]:
floating_ip1 = associate_floating_ip(server_name1)
print('Floating IP: ' + str(floating_ip1))

#### Delete Server

delete_server_by_name(server_name1)

delete_server_by_name(server_name2)

#### De-configure Network

In [ ]:
try:
    result = detach_router_by_name(router_name=router_name1, subnet_name=subnet_name1)
   # result = detach_router_by_name(router_name=router_name2, subnet_name=subnet_name2)
    #result = detach_router_by_name(router_name=router_name3, subnet_name=subnet_name3)
except Exception as e:
    print("detach_router_by_name error:" + str(e))
    pass

try:
    result = delete_router_by_name(router_name1)
    #result = delete_router_by_name(router_name2)
    #result = delete_router_by_name(router_name3)
except Exception as e:
    print("delete_router_by_name error: " + str(e))
    pass

try:
    result = delete_subnet_by_name(subnet_name1)
    #result = delete_subnet_by_name(subnet_name2)
    #result = delete_subnet_by_name(subnet_name3)
except Exception as e:
    print("delete_subnet_by_name error: " + str(e))
    pass

try:
    print("")
    #result = delete_network_by_name(network_name1)
    #result = delete_network_by_name(network_name2)
   # result = delete_network_by_name(network_name3)
except Exception as e:
    print("delete_network_by_name error: " + str(e))
    pass

#### Release Lease

delete_lease_by_name(node_lease_name)


delete_lease_by_name(net_lease_name1)